# Problem Description
Wake county located in North Carolina is one of the fastest growing counties in the USA with large influx of immigrants from other states and nationalities calling it their home.The major city Raleigh is situated in wake county has a high concentration of highly educated workforce with a good mix of young and middle aged population drives the economy of the state and opens up tremendous possibilities for new businesses to thrive.North carolina has been influenced greatly by settlers from New England and Hispanic immigrants.North carolina is also famous for its barbecue and bacon but other cuisines are starting to becoming popular. This opens up a possibility to create a recommender system for anyone who wants to open a new restaurant that can be successful in the area.We will attempt to determine a good location to open a new restaurant by exploring the neighbourhoods for restaurants distribution  within Raleigh using recently learnt data science techniques

The following datasets will be used for our assessment
Raleigh restaurants info using Foursquare
Raleigh Neighbourhoods 
1. https://en.wikipedia.org/wiki/Raleigh,_North_Carolina_neighborhoods 
2. https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Raleigh,_North_Carolina
3. Restaraunt density data from Foursquare apis
4. Classification of types of restaurants based on cuisines

## Data exploration and Presentation 
The data from the above datasets will be read via pandas dataframe and cleaned up to classify Restaurants to Neighbouhoods.Then we will be able to cluster the restaurants in the area and as well as correlate the quality of restaurants again based on cuisine to the neighbourhood which would potentially provide us insights on the type of restaurant that can be setup. We will then perform data clustering using KNN on the restaurant data and generate visualisation plots and graphs that will aid in the final decision
Recommendation Success Criteria using 

In [1]:
import requests
from bs4 import BeautifulSoup
import reverse_geocode
import folium
from geopy.geocoders import Nominatim
import numpy as np
import matplotlib as mpl
import pandas as pd
from sklearn import datasets
import geocoder
from geopy.exc import GeocoderTimedOut
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
Raleigh_Neighborhoods=[]
response_obj=requests.get('https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Raleigh,_North_Carolina').text
soup=BeautifulSoup(response_obj,'lxml')
mydivs = soup.findAll("div", {"class": "mw-category-group"})
for div in mydivs:
    for l in div.findAll('a'):
        for title in l:
            Raleigh_Neighborhoods.append(title)
Raleigh_Neighborhoods.remove('Raleigh, North Carolina neighborhoods')
Raleigh_Neighborhoods=[s.replace('Historic District','') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s.replace('(North Carolina)','') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s.replace('(Raleigh, North Carolina)','') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s.strip(' ') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s+(',Raleigh, North Carolina') for s in Raleigh_Neighborhoods]

In [3]:
df_ral=pd.DataFrame(Raleigh_Neighborhoods)
df_ral=df_ral.rename(columns={0:'Neighborhoods'})
df_ral=df_ral.drop_duplicates(keep='first')
df_ral

,Neighborhoods
0,"Battery Heights,Raleigh, North Carolina"
1,"Bloomsbury,Raleigh, North Carolina"
2,"Boylan Heights,Raleigh, North Carolina"
3,"Cameron Park,Raleigh, North Carolina"
4,"Cameron Village,Raleigh, North Carolina"
6,"Capitol Area,Raleigh, North Carolina"
7,"Capitol Heights,Raleigh, North Carolina"
8,Centennial Campus of North Carolina State Univ...
9,"Depot,Raleigh, North Carolina"
10,"East Raleigh–South Park,Raleigh, North Carolina"


In [5]:
geolocator=Nominatim(user_agent='Raleigh')
city=[]
lat=[]
long=[]
try:
    for i in Raleigh_Neighborhoods:
        location = geolocator.geocode(i)
        if location is not None:
            city.append(i)
            lat.append(location.latitude)
            long.append(location.longitude)
except GeocoderTimedOut as e:
    print("Error: geocode failed on input %s with message %s"%(location, e.msg))

In [6]:
df=pd.DataFrame({'Neighborhoods':city,'latitude':lat,'longitude':long})

In [7]:
df=df.drop_duplicates(subset='latitude',keep='first')

In [8]:
df

,Neighborhoods,latitude,longitude
0,"Battery Heights,Raleigh, North Carolina",35.777058,-78.617563
1,"Bloomsbury,Raleigh, North Carolina",35.808897,-78.648599
2,"Boylan Heights,Raleigh, North Carolina",35.774159,-78.652102
3,"Cameron Park,Raleigh, North Carolina",35.785779,-78.655470
4,"Cameron Village,Raleigh, North Carolina",35.790361,-78.660413
6,"Capitol Area,Raleigh, North Carolina",35.789884,-78.621851
8,"Depot,Raleigh, North Carolina",35.780398,-78.639099
9,"Fayetteville Street (Raleigh),Raleigh, North C...",35.768875,-78.641336
11,"Glenwood Avenue,Raleigh, North Carolina",35.886724,-78.744499
12,"Hayes Barton,Raleigh, North Carolina",35.802755,-78.646583


In [9]:
# NorthwestRaleigh,Raleigh,NC,35.8592131,-78.7820008
# North Raleigh,Raleigh,NC,35.9103757,-78.6801222,12
# NortheastRaleigh,Raleigh,NC,35.8497131,-78.6123274
# SouthwestRaleigh,Raleigh,NC,35.7673757,-78.7586629
# SoutheastRaleigh,Raleigh,NC,35.7520018,-78.6401082
# EastRaleigh,Raleigh,NC,35.7924627,-78.6189683
# raleigh=['North Raleigh,Raleigh,nc','Southwest raleigh,Raleigh,nc','Southeast raleigh,Raleigh,nc'
#         ,'east raleigh,Raleigh,nc']
# raliegh_side=[]
# lat=[]
# long=[]
# try:
#     for i in raleigh:
#         location = geolocator.geocode(i)
#         if location is not None:
#             raliegh_side.append(i)
#             lat.append(location.latitude)
#             long.append(location.longitude)
# except GeocoderTimedOut as e:
#     print("Error: geocode failed on input %s with message %s"%(location, e.msg))
# df_ral=pd.DataFrame({'Raleigh_Side':raliegh_side,'latitude':lat,'longitude':long})
# df_ral
raleigh=['Northwest Raleigh,Raleigh,NC','North Raleigh,Raleigh,NC','Northeast Raleigh,Raleigh,NC',
              'Southwest Raleigh,Raleigh,NC','Southeast Raleigh,Raleigh,NC','East Raleigh,Raleigh,NC','West Raleigh,Raleigh,NC']
raleigh_lat=[35.8592131,35.9103757,35.8497131,35.7673757,35.7520018,35.7924627,35.7868173]
raleigh_long=[-78.7820008,-78.6801222,-78.6123274,-78.7586629,-78.6401082,-78.6189683,-78.6726475]

df_ral = pd.DataFrame(list(zip(raleigh, raleigh_lat,raleigh_long)), 
               columns =['Raleigh_Side', 'latitude','longitude']) 
df_ral

,Raleigh_Side,latitude,longitude
0,"Northwest Raleigh,Raleigh,NC",35.859213,-78.782001
1,"North Raleigh,Raleigh,NC",35.910376,-78.680122
2,"Northeast Raleigh,Raleigh,NC",35.849713,-78.612327
3,"Southwest Raleigh,Raleigh,NC",35.767376,-78.758663
4,"Southeast Raleigh,Raleigh,NC",35.752002,-78.640108
5,"East Raleigh,Raleigh,NC",35.792463,-78.618968
6,"West Raleigh,Raleigh,NC",35.786817,-78.672647
